# Classification with Iris Dataset

This notebook demonstrates how to:

1. Define the model input and output schema
1. Load in data from a text file to an IDataView
1. Set up the training pipeline with data transforms
1. Choose an algorithm and append it to the pipeline
1. Train the model
1. Evaluate the model
1. Consume the model

## Install the necessary NuGet packages for training ML.NET model and plotting:

In [1]:

/* Notebook files contain both code snippets and rich text elements.
Use the "run" button in the left margin to execute each code snippet and explore ML.NET. */

#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.ML.AutoML, 0.20.0-preview.22356.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `Plotly.NET.Interactive.dll`

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [1]:

// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

## Define the model input and output schemas:

In [1]:
// Define the model input schema (which columns you will be loading in for training)
public class ModelInput
{
    [ColumnName(@"Label"), LoadColumn(0)]
    public string Label { get; set; }
    
    [ColumnName(@"Sepal length"), LoadColumn(1)]
    public float Sepal_length { get; set; }
    
    [ColumnName(@"Sepal width"), LoadColumn(2)]
    public float Sepal_width { get; set; }
    
    [ColumnName(@"Petal length"), LoadColumn(3)]
    public float Petal_length { get; set; }
    
    [ColumnName(@"Petal width"), LoadColumn(4)]
    public float Petal_width { get; set; }
    
}


// Define the model output schema (what the model will return)
public class ModelOutput
{
    [ColumnName("PredictedLabel")]
    public string PredictedLabel { get; set;}

    public float[] Score { get; set;}
}



## Create MLContext and load training data:

In [1]:
// Create a new MLContext (the starting point for all ML.NET operations)
var mlContext = new MLContext();

// Define path to training data
string trainValidateDataPath = @".\data\iris-train.tsv";
string testDataPath = @".\data\iris-test.tsv";

// Load data from a text file to an IDataView (a flexible, efficient way of describing tabular data)
IDataView trainValidateData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: trainValidateDataPath,
    hasHeader: true ,
    separatorChar: '\t',
    allowQuoting: true,
    allowSparse: false);

IDataView testData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: testDataPath,
    hasHeader: true ,
    separatorChar: '\t',
    allowQuoting: true,
    allowSparse: false);

// Display training data schema
display(trainValidateData.Schema); 


index Name Index IsHidden Type Annotations 0 Label 0 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 1 Sepal length 1 False RawType System.Single Schema [ ] 2 Sepal width 2 False RawType System.Single Schema [ ] 3 Petal length 3 False RawType System.Single Schema [ ] 4 Petal width 4 False RawType System.Single Schema [ ]

In [1]:
display(h4("Showing 5 rows from training DataView:"));
var fewRows = mlContext.Data.CreateEnumerable<ModelInput>(trainValidateData, reuseRowObject: false)
                    .Take(5)
                    .ToList();
display(fewRows);

Showing 5 rows from training DataView:

index,Label,Sepal_length,Sepal_width,Petal_length,Petal_width
0,setosa,5.4,3.7,1.5,0.2
1,setosa,4.8,3.4,1.6,0.2
2,setosa,4.8,3,1.4,0.1
3,setosa,4.3,3,1.1,0.1
4,setosa,5.8,4,1.2,0.2


## Create the training pipeline, choose an algorithm, and train the model:

In [1]:
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;
using Microsoft.ML.Trainers;
using Microsoft.ML;

var trainValidateSplit = mlContext.Data.TrainTestSplit(trainValidateData, 0.2);

var trainDataset = trainValidateSplit.TrainSet;
var validateDataset = trainValidateSplit.TestSet;

// Append the trainer to the data processing pipeline
var pipeline = mlContext.Transforms.ReplaceMissingValues(new []{new InputOutputColumnPair(@"Sepal length", @"Sepal length"),new InputOutputColumnPair(@"Sepal width", @"Sepal width"),new InputOutputColumnPair(@"Petal length", @"Petal length"),new InputOutputColumnPair(@"Petal width", @"Petal width")})      
                 .Append(mlContext.Transforms.Concatenate(@"Features", new []{@"Sepal length",@"Sepal width",@"Petal length",@"Petal width"}))      
                 .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName:@"Label",inputColumnName:@"Label"))   
				 .Append(mlContext.Auto().MultiClassification(labelColumnName: "Label"))
                 .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName:@"PredictedLabel",inputColumnName:@"PredictedLabel"));


var monitor = new NotebookMonitor();

var experiment = mlContext.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(50)
                    .SetDataset(trainDataset, validateDataset)
                    .SetEvaluateMetric(MulticlassClassificationMetric.MacroAccuracy, "Label", "PredictedLabel")
                    .SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var res = await experiment.RunAsync();


var model = res.Model;


index,Trial,Metric,Trainer,Parameters
0,0,0.93333334,SdcaMaximumEntropyMulti=>Unknown,"{""0"":{},""1"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
1,1,0.33333334,SdcaMaximumEntropyMulti=>Unknown,"{""0"":{},""1"":{""L1Regularization"":2.0347197,""L2Regularization"":0.9911041,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
2,2,0.8095238,SdcaLogisticRegressionOva=>Unknown,"{""0"":{},""1"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
3,3,1,LbfgsLogisticRegressionOva=>Unknown,"{""0"":{},""1"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
4,4,0.8,FastForestOva=>Unknown,"{""0"":{},""1"":{""NumberOfTrees"":4,""NumberOfLeaves"":4,""FeatureFraction"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
5,5,0.93333334,FastTreeOva=>Unknown,"{""0"":{},""1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
6,6,1,LbfgsMaximumEntropyMulti=>Unknown,"{""0"":{},""1"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
7,7,0.93333334,LightGbmMulti=>Unknown,"{""0"":{},""1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""LearningRate"":1,""NumberOfTrees"":4,""SubsampleFraction"":1,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
8,8,1,SdcaMaximumEntropyMulti=>Unknown,"{""0"":{},""1"":{""L1Regularization"":0.4914682,""L2Regularization"":0.03125,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"
9,9,0.33333334,SdcaLogisticRegressionOva=>Unknown,"{""0"":{},""1"":{""L1Regularization"":6.023176,""L2Regularization"":0.49239713,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""2"":{}}"


## Evaluate the model:

In [1]:
// Evaluate the model using the cross validation method
// Learn more about cross validation at https://aka.ms/mlnet-cross-validation


var testDataPredictions = model.Transform(testData);
MulticlassClassificationMetrics trainedModelMetrics = mlContext.MulticlassClassification.Evaluate(testDataPredictions);


trainedModelMetrics

: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#26.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Consume the model

In [1]:
 // Define sample model input
var sampleData = new ModelInput()
{
    Sepal_length = 4.8F,
    Sepal_width = 3.4F,
    Petal_length = 1.6F,
    Petal_width = 0.2F,
};

// Create a Prediction Engine (used to make single predictions)
var predEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
// Use the model and Prediction Engine to predict on new sample data
var predictionResult = predEngine.Predict(sampleData);
Console.WriteLine("Using model to make single prediction -- Comparing actual Label with predicted Label from sample data...\n\n");

Console.WriteLine($"Label: {0F}");
Console.WriteLine($"Sepal_length: {4.8F}");
Console.WriteLine($"Sepal_width: {3.4F}");
Console.WriteLine($"Petal_length: {1.6F}");
Console.WriteLine($"Petal_width: {0.2F}");

Console.WriteLine($"\n\nPredicted Label: {predictionResult.PredictedLabel}\n\n");

: System.ArgumentNullException: Value cannot be null. (Parameter 'transformer')
   at Microsoft.ML.Runtime.Contracts.CheckValue[T](IExceptionContext ctx, T val, String paramName)
   at Microsoft.ML.PredictionEngineBase`2.TransformerChecker(IExceptionContext ectx, ITransformer transformer)
   at Microsoft.ML.PredictionEngineBase`2..ctor(IHostEnvironment env, ITransformer transformer, Boolean ignoreMissingColumns, SchemaDefinition inputSchemaDefinition, SchemaDefinition outputSchemaDefinition, Boolean ownsTransformer)
   at Microsoft.ML.PredictionEngine`2..ctor(IHostEnvironment env, ITransformer transformer, Boolean ignoreMissingColumns, SchemaDefinition inputSchemaDefinition, SchemaDefinition outputSchemaDefinition, Boolean ownsTransformer)
   at Microsoft.ML.PredictionEngineExtensions.CreatePredictionEngine[TSrc,TDst](ITransformer transformer, IHostEnvironment env, Boolean ignoreMissingColumns, SchemaDefinition inputSchemaDefinition, SchemaDefinition outputSchemaDefinition, Boolea